In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import itertools
import random
import os
from tensorflow.keras.applications import MobileNetV2,EfficientNetV2B0
from tensorflow.keras import layers, models, Sequential, callbacks, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.metrics import precision_score, recall_score

# Ensure TensorFlow and CuDNN operations are deterministic
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Ensure using the correct GPU
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"


In [ ]:
# Set random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
tf.keras.utils.set_random_seed(seed)  # Set seed for Keras

In [ ]:
NUM_CLASSES = 2
TARGET_HEIGHT = 244
TARGET_WIDTH = 244
BATCH_SIZE = 16
EPOCHS = 500
IMAGE_DIMENSIONS = (244,244)

#add the path to the dataset
train_dir = 'smoking2/Training/Training'
val_dir = 'smoking2/Validation/Validation'
test_dir = 'smoking2/Testing'

In [ ]:
def preprocess(dataset):
    
    def preprocess_image(image, label):
        # Convert image to float32
        image = tf.image.convert_image_dtype(image, tf.float32)
        # Resize image
        image = tf.image.resize(image, [TARGET_HEIGHT, TARGET_WIDTH])
        # Apply random contrast with a chosen range
        image = tf.image.random_contrast(image, lower = 0.50, upper = 1.50)
        image = tf.image.random_saturation(image, lower = 0.50, upper = 1.50)
        image = tf.image.random_hue(image, 0.2)
        image = tf.image.random_flip_left_right(image)

        # Normalize to keep pixel values between 0 and 1
        image_min = tf.reduce_min(image)
        image_max = tf.reduce_max(image)
        
        # Scale the image to [0, 1] if the max-min > 0 to avoid division by zero
        image = tf.cond(
            tf.greater(image_max - image_min, 0),
            lambda: (image - image_min) / (image_max - image_min),
            lambda: image  # If the image is constant, just return it as is
        )

        #################################
        image = tf.keras.applications.efficientnet_v2.preprocess_input(image)

        
        return image, label
        
    # Preprocess images, cache, and prefetch
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Load datasets
train_dataset = image_dataset_from_directory(train_dir, image_size=IMAGE_DIMENSIONS, batch_size=BATCH_SIZE, label_mode='binary', shuffle=True, seed=seed)
validation_dataset = image_dataset_from_directory(val_dir, image_size=IMAGE_DIMENSIONS, batch_size=BATCH_SIZE, label_mode='binary', shuffle=False, seed=seed)
test_dataset = image_dataset_from_directory(test_dir, image_size=IMAGE_DIMENSIONS, batch_size=BATCH_SIZE, shuffle=False, seed=seed)

class_names = train_dataset.class_names

# Preprocess the datasets
train_dataset = preprocess(train_dataset)
validation_dataset = preprocess(validation_dataset)
test_dataset = preprocess(test_dataset)

In [ ]:
def count_labels(dataset, alpha=0.60, scaling_strategy='log', base_multiplier=0.50):
    
    label_counts = Counter()
    
    # Count label occurrences batch-wise
    for _, labels in dataset:  
        label_indices = np.argmax(labels.numpy(), axis=1)  # Convert one-hot to label indices
        label_counts.update(label_indices)
    
    # Create a list of labels and their counts
    all_labels = np.concatenate([y.numpy() for _, y in train_dataset.take(-1)]).flatten()
    all_counts = [count for label, count in label_counts.items()]
    
    # Compute base class weights (inverse proportional to class frequency)
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(all_labels), 
                                         y=all_labels)
    
    # Apply custom scaling strategies
    if scaling_strategy == 'power':
        # Raise weights to a power to control the effect of imbalance
        class_weights = np.power(class_weights, alpha)
    elif scaling_strategy == 'log':
        # Apply logarithmic scaling to the class weights
        class_weights = np.log1p(class_weights)
    
    # Multiply by the base multiplier
    class_weights *= base_multiplier
    
    # Convert class weights to a dictionary format
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    
    return class_weights_dict

# Count the labels in the dataset
class_weights_dict = count_labels(train_dataset)

In [ ]:
print(class_weights_dict)

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
def create_efficientnet_model(num_classes=1):
    data_augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.5),  # Reduced from 0.1
        layers.RandomZoom(0.3),      # Reduced from 0.1
        layers.RandomTranslation(0.3, 0.3),  # Reduced from 0.1
        # layers.RandomContrast(0.2)
    ], name="data_augmentation")

    model = Sequential([
        tf.keras.Input(shape=(TARGET_HEIGHT, TARGET_WIDTH, 3)),
        data_augmentation,
            
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.03)),  # Increased regularization
        layers.BatchNormalization(),
        layers.Dropout(0.5),  # Increased dropout
        
        # Simplified architecture with fewer layers and neurons
        
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.03)),  # Increased regularization
        layers.BatchNormalization(),
        layers.Dropout(0.5),  # Increased dropout

        layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.03)),  # Increased regularization
        layers.BatchNormalization(),
        layers.Dropout(0.3),  # Increased dropout

        
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile model with reduced learning rate
    initial_learning_rate = 1e-4  # Reduced from 5e-4
    
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=initial_learning_rate, 
            weight_decay=0.1  # Increased weight decay
        ),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]  # Added AUC metric
    )
    
    # Return only the model, not a tuple
    return model

In [ ]:
# Callbacks
########################################
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)
lr_schedule = callbacks.LearningRateScheduler(lambda epoch: float(1e-3 * tf.math.exp(-0.04 * epoch)), verbose=1)
reduce_lr_on_plateau = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, verbose=1)
model_checkpoint = callbacks.ModelCheckpoint('model.keras', monitor='val_loss', save_best_only=True, verbose=1)

model = create_efficientnet_model(NUM_CLASSES)
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_schedule, reduce_lr_on_plateau, model_checkpoint],
    # class_weight={0: 1, 1: 1}
    )

In [ ]:
import seaborn as sns
# from metrics_calculator import calculate_metrics

def calculate_metrics(model, dataset, dataset_name="Dataset", show_predictions=False, num_examples=5):
    y_true = []
    y_pred_probs = []
    images_to_show = []
    labels_to_show = []
    probs_to_show = []

    for i, (images, labels) in enumerate(dataset):
        batch_pred_probs = model.predict(images, verbose=0)

        y_true.append(labels.numpy())
        y_pred_probs.append(batch_pred_probs)

        if show_predictions and len(images_to_show) < num_examples:
            images_to_show.extend(images.numpy())
            labels_to_show.extend(labels.numpy())
            probs_to_show.extend(batch_pred_probs)

    y_true = np.concatenate(y_true, axis=0)
    y_pred_probs = np.concatenate(y_pred_probs, axis=0)

    y_pred = (y_pred_probs > 0.5).astype(int)

    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        if y_pred.shape[1] > 1:
            y_pred = np.argmax(y_pred, axis=1)
        else:
            y_pred = y_pred.flatten()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='binary', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)

    if len(y_pred_probs.shape) > 1 and y_pred_probs.shape[1] > 1:
        y_pred_prob = y_pred_probs[:, 1]
    else:
        y_pred_prob = y_pred_probs

    print(f"\n===== {dataset_name} Metrics =====")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Smoking','Not Smoking'],
                yticklabels=['Smoking','Not Smoking'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{dataset_name} Confusion Matrix')
    plt.tight_layout()
    plt.show()

    if show_predictions:
        class_names = ['Smoking','Not Smoking']
        plt.figure(figsize=(15, 3 * num_examples))
        for idx in range(min(num_examples, len(images_to_show))):
            img = images_to_show[idx]
            true_label = labels_to_show[idx]
            prob = 1-probs_to_show[idx]
            predicted_class = (prob < 0.5).astype(int)

            plt.subplot(num_examples, 1, idx + 1)
            plt.imshow(img.squeeze(), cmap='gray' if img.shape[-1] == 1 else None)
            plt.axis('off')
            plt.title(f"Predicted: {class_names[int(predicted_class)]} "
                      f"({float(prob):.2f}) | True: {class_names[int(true_label)]}")
        plt.tight_layout()
        plt.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm
    }

In [ ]:

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)


plt.tight_layout()
plt.show()

In [ ]:
test_metrics = calculate_metrics(model, test_dataset, "Test", show_predictions=True, num_examples=40)


In [ ]:

# Read and preprocess the image
img = tf.io.read_file('/kaggle/input/smoking2/model/Training/Training/notSmoking/notsmoking_0008.jpg')
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, [TARGET_HEIGHT, TARGET_WIDTH])

# Add batch dimension
img = tf.expand_dims(img, axis=0)  # Shape becomes (1, 80, 80, 3)

# Predict
predictions = model.predict(img)
print("Raw prediction:", predictions)